In [92]:
import torch

print("cuda" if torch.cuda.is_available() else "cpu")

cuda


In [93]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

torch.manual_seed(799)
tkz = GPT2Tokenizer.from_pretrained("gpt2")
mdl = GPT2LMHeadModel.from_pretrained("gpt2")

ln = 10 # 길이
cue = "It will"
gen = tkz.encode(cue) # cue 토큰화
ctx = torch.tensor([gen]) # totensor

In [94]:
prv=None
for i in range(ln):
    op= mdl(ctx, past_key_values=prv)
    prv = op.past_key_values
    tkn = torch.argmax(op['logits'].squeeze(0))

    gen += [tkn.tolist()]
    ctx = tkn.unsqueeze(0)

seq = tkz.decode(gen)

print(seq)

It will is not a matter of whether the government will be


In [201]:
# greedy한 결과값만 취하므로 늘 똑같은 output를 생성 
ip_ids = tkz.encode(cue, return_tensors='pt')
op_greedy = mdl.generate(ip_ids, max_length=ln, pad_token_id=tkz.eos_token_id)
seq = tkz.decode(op_greedy[0], skip_special_tokens=True)
print(seq)

It will be interesting to see how the new system


In [200]:
# beam search
# num_beams : 단어 예측값을 지속적으로 곱하여 가장 높은 확률이 되는 상위 n개를 유지하며 생성
op_beam = mdl.generate(
    ip_ids, 
    max_length=ln, 
    num_beams=10, 
    num_return_sequences=3, 
    pad_token_id=tkz.eos_token_id
)

for op_beam_cur in op_beam:
    print(tkz.decode(op_beam_cur, skip_special_tokens=True))

It will be interesting to see how this plays out
It will be interesting to see how long it takes
It will be interesting to see what happens with the


In [253]:
# top-k sampling
# top-k개의 확률값을 정규화하여 랜덤하게 뽑아 선택하는 방식 
# 일정하지 않고, 다양한 결과를 얻을 수 있음
# top_k= 0 으로 설정시, 모든 단어로 샘플링 진행
for i in range(3):
    #torch.manual_seed(i)
    op = mdl.generate(
        ip_ids, 
        do_sample=True, 
        max_length=ln, 
        top_k=4,
        pad_token_id=tkz.eos_token_id
    )

    seq = tkz.decode(op[0], skip_special_tokens=True)
    print(seq)

It will not be a long-term solution.
It will take a lot of time and a lot
It will also be a great way to learn about


In [261]:
tkz.pad_token_id

In [285]:
# top-p sampling 방식
# 누적확률임계값을 지정하여 단어의 폭을 조정
# top_k 값을 변경하여 동시에 사용가능
for i in range(3):
    #torch.manual_seed(i)
    op = mdl.generate(
        ip_ids, 
        do_sample=True, 
        max_length=ln, 
        top_p=0.75, 
        top_k = 0,
        pad_token_id=tkz.eos_token_id
    )

    seq = tkz.decode(op[0], skip_special_tokens=True)
    print(seq)

It will be interesting to see if he is able
It will also provide a better understanding of how the
It will be interesting to see how the next few
